# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
import numpy as np
import pandas as pd

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [3]:
pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [4]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [6]:
pokemon.Legendary.value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [17]:
leg = pokemon[pokemon.Legendary == True]
non_leg = pokemon[pokemon.Legendary == False]

print ('Legendary Total - Mean: ',round(leg.Total.mean(),2), '  Std. Dev.: ', round(leg.Total.std(),2))
print ('Non-Legendary Total - Mean: ',round(non_leg.Total.mean(),2), '  Std. Dev.: ', round(non_leg.Total.std(),2))

Legendary Total - Mean:  637.38   Std. Dev.:  60.94
Non-Legendary Total - Mean:  417.21   Std. Dev.:  106.76


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [22]:
from scipy.stats import ttest_ind

ttest_ind(leg.Total, non_leg.Total, equal_var=False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [6]:
"""Since pvalue is very small we can reject the Null Hip. with a very high confidence level - 99.99999999...%, that
is we can reject that the mean of both populations is identical, therefore there is significant differenc between
the mean of both populations"""

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [23]:
pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ghost        32
Ground       32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [28]:
pokwater = pokemon[pokemon['Type 1'] == 'Water']
poknonwater = pokemon[pokemon['Type 1'] != 'Water']

print('Means - Water:', pokwater['Total'].mean(), 'Non Water:', poknonwater['Total'].mean())
print('St. Dev. - Water:', pokwater['Total'].std(), 'Non Water:', poknonwater['Total'].std())

Means - Water: 430.45535714285717 Non Water: 435.85901162790697
St. Dev. - Water: 113.18826606431458 Non Water: 121.09168230208066


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [29]:
ttest_ind(pokwater['Total'], poknonwater['Total'], equal_var=True)

Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [30]:
"""Since pvalue>0.05 we cannot reject the Null Hip. that the means are equal for both Water and Non- Water pokemons"""

'Since pvalue>0.05 we cannot reject the Null Hip. that the means are equal for both Water and Non- Water pokemons'

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [31]:
from scipy.stats import ttest_rel

ttest_rel(pokemon.Attack, pokemon.Defense)

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [12]:
"Null Hip rejected, so there is a significant difference between the means of Defense and Attack Scores"

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [34]:
ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [14]:
"""Null H. cannot be rejected, we cannot reject the possibility that the means of Sp. Atk and Sp. Def are the
same."""

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [36]:
from scipy.stats import ttest_1samp

ttest_1samp (pokemon.Attack-pokemon.Defense,0)

Ttest_1sampResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [40]:
cross = pd.crosstab(pokemon.Legendary, pokemon['Type 1'] == 'Water')
cross

Type 1,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [44]:
from scipy.stats import chi2_contingency

chi = chi2_contingency(cross)
chi

(2.9429200762850503, 0.08625467249550949, 1, array([[632.1, 102.9],
        [ 55.9,   9.1]]))

Based on a 95% confidence, should we reject the null hypothesis?

In [46]:
p_value = chi [1]

0.08625467249550949

In [ ]:
"""Since p_value > 0.05, we cannot reject the Null H. that the variables are dependant"""